In [1]:
import json

import numpy as np
import pandas as pd

In [ ]:
with open('../data/source/events-00001.json', "r") as file:
    file = json.load(file)

In [4]:
events = set()
concepts = set()
counts = []

for event in file:
    events.add(event['info']['uri'])
    counts.append(event['info']['articleCounts']['total'])
    # for similar_event in event['similarEvents']['similarEvents']:
    #     events.add(similar_event['uri'])
    for concept in event['info']['concepts']:
        concepts.add(concept['uri'])

In [5]:
len(events)

1000

In [6]:
len(concepts)

8729

In [52]:
counts = np.array(counts)

In [54]:
counts[counts > 0].shape

(608,)

In [53]:
np.unique(counts[counts > 0], return_counts=True)

(array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  35,  37,  39,  40,  45,  46,
         48,  51,  52,  56,  57,  58,  60,  62,  69,  79,  89,  92,  94,
        100, 111, 121, 122, 123, 131, 137, 141, 144, 150, 152, 164, 170,
        186, 221, 233, 313, 335, 394, 431, 447, 607]),
 array([97, 61, 44, 41, 38, 37, 33, 19, 30, 21, 25, 22, 16, 12,  6,  4,  7,
         4,  3,  7,  1,  5,  2,  4,  2,  2,  4,  1,  1,  1,  3,  2,  3,  3,
         4,  1,  1,  1,  1,  2,  1,  1,  1,  1,  1,  1,  3,  2,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1]))

# Preprocessed

In [ ]:
import pickle

with open('../data/graphs/events.pkl', 'rb') as file:
    events = pickle.load(file)